# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support,classification_report
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection  import GridSearchCV
nltk.download(['punkt','stopwords','wordnet'])

In [ ]:
# load data from database
engine = create_engine('sqlite:///message_all.db')
df=pd.read_sql('SELECT * from message_all', engine)
X = df['message']
# result is multiple classifty.
y = df.iloc[:,4:]

In [ ]:
def load_data(db_path='workspace/data/DisasterResponse.db',tablename='disastertab'):
    """
    Function: load data from database and return X and y.
    Args:
      db_path(str): database file name included path
      tablename:(str): table name in the database file.
    Return:
      X(pd.DataFrame): messages for X
      y(pd.DataFrame): labels part in messages for y
    """
    # load data from database
    engine = create_engine('sqlite:///'+db_path)
    df=pd.read_sql('SELECT * from '+tablename, engine)
    X = df['message']
    # result is multiple classifty.
    y = df.iloc[:,4:]
    return X, y

In [ ]:
X,y=load_data()

In [ ]:
print(X[:5].values.tolist())
#need sent_tokenize to parse X.

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    """
    Function: tokenize the text
    Args:  source string
    Return:
    clean_tokens(str list): clean string list
    
    """
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:


def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    #accuracy = accuracy_score(y_test,y_pred)
    report=classification_report(y_test, y_pred,target_names=y_test.columns.values)
    #label_ranking_average_precision =[0,1] , the higher ,the better.
    avg_precision = label_ranking_average_precision_score(y_test, y_pred)

    print('label ranking average precision: {}'.format(avg_precision))
    print('The metrics for each output category')
    print(report)
    

In [ ]:
y_pred=pipeline.predict(X_test)

In [ ]:
print(y_pred.shape)
print(y_test.shape)
tt=pd.DataFrame(y_pred,columns=y_test.columns.values)
tt.max()
#tt.loc[tt['related']==2,'related'] = 1
#y_pred=tt.values

In [ ]:
(y.related==2).sum()

In [ ]:
display_results(y_test, y_pred)

In [ ]:
pipeline.get_params()

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__estimator__criterion':['gini', 'entropy'],
    'vect__smooth_idf': [True,False],
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)

parameters_bak = {
    'clf__estimator__n_estimators': [200,100],
    'clf__estimator__max_depth': [3,15],
    'clf__estimator__min_samples_leaf': [1,8],
    'vect__smooth_idf': [True,False],
    'vect__sublinear_tf':[True,False]
}

In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)   

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
display_results(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf', AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=2), n_estimators = 4))
])

In [ ]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf', AdaBoostClassifier(base_estimator =MultinomialNB()))
])

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'vect__smooth_idf': [True,False],
}
# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)   
display_results(y_test, y_pred)

### 9. Export your model as a pickle file

In [ ]:
with open('classifer.pkl', 'wb') as f:
    pickle.dump(cv, f)


In [ ]:
    parameters = {
        'clf__estimator__criterion':['gini', 'entropy']
        'vect__smooth_idf': [True,False],
    }

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sys
import re
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import label_ranking_average_precision_score,classification_report
from sklearn.model_selection  import GridSearchCV
nltk.download(['punkt','stopwords','wordnet'])

def load_data(db_path='workspace/data/DisasterResponse.db',tablename='disastertab'):
    """
    Function: load data from database and return X and y.
    Args:
      db_path(str): database file name included path
      tablename:(str): table name in the database file.
    Return:
      X(pd.DataFrame): messages for X
      y(pd.DataFrame): labels part in messages for y
    """
    # load data from database
    engine = create_engine('sqlite:///'+db_path)
    df=pd.read_sql('SELECT * from '+tablename, engine)
    X = df['message']
    # result is multiple classifty.
    y = df.iloc[:,4:]
    return X, y

def tokenize(text):
    """
    Function: tokenize the text
    Args:  source string
    Return:
    clean_tokens(str list): clean string list
    
    """
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

def build_model(): 
    """
    Function: build model that consist of pipeline
    Args:
      N/A
    Return
      cv(model): Grid Search model 
    """
    
    pipeline = Pipeline([
        ('vect',TfidfVectorizer(stop_words='english',tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100,random_state=20)))
    ]) 
    parameters = {
        'clf__estimator__criterion':['entropy']
    }
    
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
    return cv

def evaluate_model(y_test, y_pred):
    report=classification_report(y_test, y_pred,target_names=y_test.columns.values)
    avg_precision = label_ranking_average_precision_score(y_test, y_pred)
    print('label ranking average precision: {}'.format(avg_precision))
    print('The metrics for each output category')
    print(report)
def save_model(cv):
    """
    Function: save model as pickle file.
    Args:
      cv:target model
    Return:
      N/A
    """
    with open('classifer.pkl', 'wb') as f:
        pickle.dump(cv, f)    
def main(): 
    print("Load data")
    X,y=load_data()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
    print("Build model")
    model=build_model()
    print("Train model")
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print("Evaluation model")
    evaluate_model(y_test, y_pred)
    print("Save model")
    save_model(model)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\binwan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\binwan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\binwan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
if __name__ == "__main__":
    main()

Load data
Build model
Train model


In [ ]:
def build_model2(): 
    """
    Function: build model that consist of pipeline
    Args:
      N/A
    Return
      cv(model): Grid Search model 
    """
    
    pipeline = Pipeline([
        ('vect',TfidfVectorizer(tokenizer=tokenize)),
        ('clf', AdaBoostClassifier(base_estimator =MultinomialNB()))
    ]) 
    parameters = {
        'clf__alpha':[1.0]
        #'vect__smooth_idf':[True] 
    }
    
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
    return cv
def main2(): 
    print("Load data")
    X,y=load_data()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
    print("Build model")
    model=build_model2()
    print("Train model")
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print("Evaluation model")
    evaluate_model(y_test, y_pred)
    print("Save model")
    save_model(model)

In [ ]:
main2()